In [1]:
import numpy as np
from twitter_scraper import get_tweets
from twitterscraper import query_tweets
import re
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
tweets_info = query_tweets('from:realDonaldTrump', begindate=dt.date(2018,4,11), enddate=dt.date.today())
    # def query_tweets(query, limit=None, begindate=dt.date(2017,1,1), enddate=dt.date.today(), poolsize=20, lang=''):
    
tweets = []
for tweet_info in tweets_info:
    # text = re.sub(r'^https?:\/\/.*[\r\n]*', '', tweet.text, flags=re.MULTILINE)
    tweet = tweet_info.text
    tweets.append(tweet)
#     joined_text += tweet.text + ' bbroy '

tweets = np.array(tweets)

# print(tweets)

In [38]:
tweets_info = get_tweets('realDonaldTrump', pages = 10)
tweets = []
for tweet_info in list(tweets_info):
    text = tweet_info['text']
    # Add NEWTWEET to help recognize starting word of tweet for future processing
    tweets.append("NEWTWEET " + text)
    
tweets = np.array(tweets)
num_tweets = len(tweets)
print("Number of tweets: " + str(num_tweets))
# print(tweets)

Number of tweets: 200


In [39]:
words = []
for tweet in tweets:
    words.extend(tweet.split(" "))
    
cleaned_words = []
for word in words:
    if ".com" not in word and "www." not in word and "http" not in word and word != '':
        cleaned_words.append(word)
        
words = cleaned_words
cleaned_words_no_repeat = [words[0]] # could be repeated 'NEWTWEET' due to deleting URL's
        
for i in range(1, len(words)):
    if not (words[i] == "NEWTWEET" and words[i] == words[i-1]):
        cleaned_words_no_repeat.append(words[i])
        
words = cleaned_words_no_repeat
words = np.array(words)
# print(words)

In [40]:
G = nx.DiGraph()
# Initialize states with start and end states (assuming no word is 'S' or 'E')
states = ['S', 'E']
for word in words:
    if word not in states and word != "" and word != "NEWTWEET":
        states.append(word)
G.add_nodes_from(states)
# nx.draw(G, with_labels = True)

In [41]:
freq = {}
edges = []

# Edge case for first word in tweet not being connected to Start state

for i in range(len(words) - 1):
    if words[i] == "NEWTWEET":
        edges.append(('S', words[i + 1], 1))
        continue
    elif words[i+1] == "NEWTWEET":
        edges.append((words[i], 'E', 1))
        continue
    elif (words[i], words[i+1]) in freq:
        freq[(words[i], words[i+1])] += 1
    else:
        freq[(words[i], words[i+1])] = 1
        
    if words[i].endswith(('.', '!', '?')):
#         print(words[i], words[i+1])
        edges.append((words[i], 'E', 1))
        edges.append(('S', words[i + 1], 1))
        # remove 'word. Word' so no edge is made between them?
        
edges.append(('E', 'S', 1))

for edge in freq:
    edges.append((edge[0], edge[1], freq[edge]))
    
G.add_weighted_edges_from(edges)
# nx.draw(G, with_labels = True)

In [42]:
# print(list(G.successors('S')))
# print(list(G.predecessors('E')))

# print(list(G.predecessors('S')))
# print(list(G.successors('E')))

In [43]:
# Creates numpy adjacency matrix of states, where Starting node is row 0 and Ending node is row 1
matrix = nx.to_numpy_matrix(G, nodelist = states)
matrix = np.array(matrix).tolist()
# print(matrix)

In [44]:
# Sanity check to see if adjacency matrix reflects actual state of MC
num = 0
for i in range(len(matrix[0])):
    num += matrix[0][i]
print(num == len(list(G.successors('S'))))

True
